In [4]:
# Load packages and directories


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import os
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import seaborn as sns
import importlib

import preparing_for_bill as pfb

import re
import pickle 
from concurrent.futures import ProcessPoolExecutor
base_directory = "C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning"
data_directory = os.path.join(base_directory, 'edp_data')
cleaned_data_directory = os.path.join(base_directory, 'resampled_30min_data')

# Directory containing the household profile CSV files
survey_data = pd.read_csv(os.path.join(base_directory, 'survey/edp_survey_aircon.csv'))  
output_directory = os.path.join(base_directory, 'figures')  # Change this to your output directory
resampled_data_directory = os.path.join(base_directory, 'resampled_30min_data_real')

filtered_data_directory = os.path.join(base_directory, 'archive/filtered_data_real')

# Directory to save profiles
site_profiles_directory = os.path.join(base_directory, 'full_profiles_real')

resampled_data_directory = os.path.join(base_directory,'resampled_30min_data_real')

In [12]:
with open("testing_site_ids.pkl", "rb") as file:
    testing_site_ids = pickle.load(file)

In [9]:

# if needed, pre-load edp raw files 

# Define directories
data_directory = os.path.join(base_directory, "edp_data")
site_profiles_directory = os.path.join(base_directory, "full_profiles_real")

# Get list of CSV files
filepaths = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory) if filename.endswith('.csv')]

# Process files one by one with optimized memory usage
dfs = []
for filepath in filepaths:
    try:
        print("reading ", filepath)
        # Read only required columns and convert `real_energy` to float32
        df = pd.read_csv(filepath, usecols=['edp_site_id', 'datetime', 'circuit_label', 'real_energy'], 
                         dtype={'real_energy': 'float32'}, 
                         engine='python', on_bad_lines='skip')

        dfs.append(df)
    except Exception as e:
        print(f"Error processing {filepath}: {e}")


# Concatenate all processed DataFrames
if dfs:
    full_data = pd.concat(dfs, ignore_index=True)
    print("Successfully loaded all files into a single DataFrame.")
else:
    print("No valid data to process.")



reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_01.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_02.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_03.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_04.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_05.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_06.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_07.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_08.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\edp_data_2023_09.csv
reading  C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\edp_data\e

In [11]:
importlib.reload(pfb)
# all 2023 site ids have been through initial processing, but use this code if not yet processed.
to_process_ids = None
pfb.process_site_2(to_process_ids, full_data, site_profiles_directory)

processing:  S0576
Processed data for site S0576 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\S0576_profile.csv
processing:  W0236
Processed data for site W0236 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\W0236_profile.csv
processing:  S0177
Processed data for site S0177 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\S0177_profile.csv
processing:  W0087
Processed data for site W0087 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\W0087_profile.csv
processing:  W0058
Processed data for site W0058 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\W0058_profile.csv
processing:  W0334
Processed data for site W0334 saved to C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning\full_profiles_real\W0334_profile.csv
processing:  W0031
Processed data for site W0031 sav

In [13]:


pfb.resample_site_profiles(testing_site_ids)

Successfully processed S0576 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0236 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed S0177 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0087 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0058 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0334 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0031 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0142 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed S0209 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0328 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed S0463 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed W0241 with columns: ['kWh', 'Air_Conditioner_Load', 'PV']
Successfully processed S0299

After Processing, Create new DF with bootstrapped gross profile. 

In [35]:
bootstraped_real_merged_df = pd.read_csv(os.path.join(base_directory, 'bootstrapped_real_merged_df.csv'))

In [36]:

print(bootstraped_real_merged_df.dtypes)

datetime               object
S0576_bootstrapped    float64
S0576_real            float64
W0236_bootstrapped    float64
W0236_real            float64
                       ...   
W0342_real            float64
W0011_bootstrapped    float64
W0011_real            float64
W0073_bootstrapped    float64
W0073_real            float64
Length: 95, dtype: object


In [38]:
importlib.reload(pfb)
pfb.add_bootstrap_to_resampled_profiles(
    base_directory,
    profiles_directory='resampled_30min_data_real',
    bootstrapped_file='bootstrapped_real_merged_df.csv', 
    output_directory='resampled_30min_data_bootstrapped_cons'
)


Processing site: S0177
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0177_processed_profile.csv

Processing site: S0179
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0179_processed_profile.csv

Processing site: S0209
Found 5 PV values below standby consumption (-0.03kW)
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0209_processed_profile.csv

Processing site: S0216
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0216_processed_profile.csv

Processing site: S0227
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0227_processed_profile.csv

Processing site: S0229
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0229_processed_profile.csv

Processing site: S0249
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0249_processed_profile.csv

Processing site: S0250
Processed data saved to: resampled_30min_data_bootstrapped_cons\S0250_processed_profile.csv

Processing site: 

In [39]:
def create_aircon_comparison_plot(df):
    """
    Creates an interactive line plot comparing gross_cons_bootstrapped_aircon and bootstrapped_aircon.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'timestamp', 'gross_cons_bootstrapped_aircon', and 'bootstrapped_aircon' columns
    
    Returns:
    plotly.graph_objects.Figure: Interactive Plotly figure
    """
    # Create figure
    fig = go.Figure()
    
    # # Add traces
    # fig.add_trace(
    #     go.Scatter(
    #         x=df['TS'],
    #         y=df['gross_cons_bootstrapped_aircon'],
    #         name='Gross Consumption',
    #         line=dict(color='#8884d8', width=2),
    #         mode='lines'
    #     )
    # )
    
    fig.add_trace(
        go.Scatter(
            x=df['TS'],
            y=df['aircon_bootstrapped'],
            name='Bootstrapped Aircon',
            line=dict(color='#82ca9d', width=2),
            mode='lines'
        )
    )
    
    # Update layout
    fig.update_layout(
        title='Aircon Consumption Comparison',
        xaxis_title='Timestamp',
        yaxis_title='Consumption',
        hovermode='x unified',
        template='plotly_white',
        showlegend=True,
        height=600,
        width=1000,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # Update x-axis
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    
    # Update y-axis
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    
    return fig

In [40]:
df_W0172 = pd.read_csv(os.path.join(base_directory, 'resampled_30min_data_bootstrapped_cons/W0166_processed_profile.csv'))
create_aircon_comparison_plot(df_W0172)